In [1]:
import numpy as np 
import pickle 

import matplotlib.pyplot as plt 
%matplotlib inline 

# test function

In [80]:
#

SAMPLE_SIZE = 768*5; NOISE_VAR = 0.0025; # sample size and noise variance

Ns = [2,4,8] # dimensions
DELTAs = [0.05,0.1,0.2] # deltas

In [81]:
# test functions 


def func1( x, sigma_sq, n): 
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) )
    
#     A = np.linspace(1,n**2, n)
    
    x = np.sum( np.cos( x ) ) + np.exp( x[0] * x[1]  ) 
    
    return x + noise 
    
    
def func2( x, sigma_sq, n): 
    
    noise = np.random.normal(0, np.sqrt( sigma_sq ) )
        
    x = np.sum( np.sin(x ) ) + np.sum( np.cos(x - np.array( [1]* n ) ) ) + np.exp( x[0] * x[1] + x[1] ) 
    
    return x + noise 
    

# In Euclidean space

## our method

In [82]:
def new_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    avg_est = 0 
    
    for _ in range(m): 

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        v = v / np.linalg.norm(v) 
        
        w = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        w = w / np.linalg.norm(w) 
                
        f1 = func( delta*v + delta*w, noise_var, n ) 
        f2 = func( delta*v - delta*w, noise_var, n ) 
        f3 = func( -delta*v + delta*w, noise_var, n ) 
        f4 = func( -delta*v - delta*w, noise_var, n ) 

        grad_est = (f1 - f2 - f3 + f4) / ( 8 * delta**2 ) * n**2 * ( np.outer(v,w) + np.outer(w,v) ) 

        avg_est = avg_est + grad_est  

    avg_est = avg_est / m 
    
    return avg_est 

## Stein's method

In [83]:
def steins_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    avg_est = 0 
    
    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
                
        f1 = func( delta*v, noise_var, n ) 
        f2 = func( -delta*v, noise_var, n ) 
        f3 = func( np.zeros(n), noise_var, n ) 
        
        D2 = (f1 - 2*f3 + f2) / (2*delta**2) 

        grad_est = ( np.outer(v,v) - np.identity(n) ) * D2 

        avg_est = avg_est + grad_est  
        
    avg_est = avg_est / m 
    
    return avg_est

## entry-wise est

In [84]:
def entry_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    H_est = np.zeros((n,n))
    
    for i in range(n): 
        for j in range(n):
            
            avg_est = 0
            
            for _ in range(m):
                
                I = delta * np.array( [1 if k==i else 0 for k in range(n) ] )
                J = delta * np.array( [1 if k==j else 0 for k in range(n) ] )
            
                f1 = func( I + J, noise_var, n ) 
                f2 = func( I - J, noise_var, n ) 
                f3 = func( - I + J, noise_var, n ) 
                f4 = func( - I - J, noise_var, n ) 

                grad_est = (f1 - f2 - f3 + f4) / ( 4 * delta**2 ) 
                
                avg_est = avg_est + grad_est  

            avg_est = avg_est / m 
            
            H_est[i,j] = avg_est 
    
    return H_est 

In [85]:
def res(n, delta, true_hess, func, noise_var = 1, m = 100, N = 100 ): 

    new_errors = [] 
    stein_errors = [] 
    entry_errors = [] 

    for _ in range(N): 
        
        new_errors.append( np.linalg.norm( new_est( func, n, noise_var, delta, int(m / 4. ) ) - true_hess , ord = 2 ) )
        
        stein_errors.append( np.linalg.norm( steins_est( func, n, noise_var, delta/np.sqrt(n), int(m/3.) ) - true_hess , ord = 2 ) )
        
        entry_errors.append( np.linalg.norm( entry_est( func, n, noise_var, delta, int(m/4./n/n) ) - true_hess , ord = 2 ) )
            
    return new_errors, stein_errors, entry_errors  

In [86]:

for n in Ns:
    
    for delta in DELTAs: 
    
        true_hess = entry_est( func1, n, 0, delta = 0.001 , m = 1 ) # estimate the true Hessian matrix (no noise here)

        new_errors, stein_errors, entry_errors = res(n, delta, true_hess, func1, noise_var = NOISE_VAR, m = SAMPLE_SIZE, N = 100 ) 
                
        pickle.dump( (new_errors, stein_errors, entry_errors), open(  'euc_res_delta{0}_n{1}'.format(delta,n), 'wb' ) ) 

# sphere

In [87]:
def exp_map(v, n ): 
    
    if (v == 0).all(): 
        return np.zeros(len(v) + 1) 
    else: 
        v = np.array(list(v) + [ 1 - np.sqrt( 1 - np.sum(v**2) ) ])
        return v
#         np.array(  [0] * (n-1) + [1] ) * np.cos( np.linalg.norm( v ) ) \
#                     + np.sin( np.linalg.norm( v ) ) * v / np.linalg.norm( v ) 

## our method

In [88]:
def new_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    avg_est = 0 
    
    for _ in range(m): 

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        v = v / np.linalg.norm(v) 
        
        w = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        w = w / np.linalg.norm(w) 
                
        f1 = func( exp_map( delta*v + delta*w, n ), noise_var, n + 1 ) 
        f2 = func( exp_map( delta*v - delta*w, n ), noise_var, n + 1 ) 
        f3 = func( exp_map( -delta*v + delta*w, n ), noise_var, n + 1 ) 
        f4 = func( exp_map( -delta*v - delta*w, n ), noise_var, n + 1 ) 

        grad_est = (f1 - f2 - f3 + f4) / ( 8 * delta**2 ) * n**2 * ( np.outer(v,w) + np.outer(w,v) ) 

        avg_est = avg_est + grad_est  

    avg_est = avg_est / m 
    
    return avg_est 

## Stein's method

In [89]:
def steins_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    avg_est = 0 
    
    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
                
        f1 = func( exp_map( delta*v, n ), noise_var, n + 1 ) 
        f2 = func( exp_map( -delta*v, n ), noise_var, n + 1 ) 
        f3 = func( exp_map( np.zeros(n), n), noise_var, n + 1 ) 
        
        D2 = (f1 - 2*f3 + f2) / (2*delta**2) 

        grad_est = ( np.outer(v,v) - np.identity(n) ) * D2 

        avg_est = avg_est + grad_est 
        
    avg_est = avg_est / m 
    
    return avg_est

## entry-wise est

In [90]:
def entry_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    H_est = np.zeros((n,n))
    
    for i in range(n): 
        for j in range(n):
            
            avg_est = 0
            
            for _ in range(m):
                
                I = delta * np.array( [1 if k==i else 0 for k in range(n) ] )
                J = delta * np.array( [1 if k==j else 0 for k in range(n) ] )
                
                f1 = func( exp_map( I + J, n ) , noise_var, n + 1 ) 
                f2 = func( exp_map( I - J, n ), noise_var, n + 1 ) 
                f3 = func( exp_map( - I + J, n ), noise_var, n + 1 ) 
                f4 = func( exp_map( - I - J, n ), noise_var, n + 1 ) 

                hess_est = (f1 - f2 - f3 + f4) / ( 4 * delta**2 ) 
                
                avg_est = avg_est + hess_est  

            avg_est = avg_est / m 
            
            H_est[i,j] = avg_est 
    
    return H_est 

In [91]:
def res(n, delta, true_hess, func, noise_var = 1, m = 100, N = 100 ): 

    new_errors = [] 
    stein_errors = [] 
    entry_errors = [] 

    for _ in range(N): 
        
        new_errors.append( np.linalg.norm( new_est( func, n, noise_var, delta, int(m / 4. ) ) - true_hess , ord = 2 ) )
        
        stein_errors.append( np.linalg.norm( steins_est( func, n, noise_var, delta/np.sqrt(n), int(m/3.) ) - true_hess , ord = 2 ) )
        
        entry_errors.append( np.linalg.norm( entry_est( func, n, noise_var, delta, int(m/4./n/n) ) - true_hess , ord = 2 ) )
            
    return new_errors, stein_errors, entry_errors 

In [92]:

for n in Ns: 
    
    for delta in DELTAs: 
    
        true_hess = entry_est( func1, n , 0, delta = 0.001 , m = 1 ) 

        new_errors, stein_errors, entry_errors = res(n , delta, true_hess, func1, noise_var = NOISE_VAR, m = SAMPLE_SIZE, N = 100 ) 
        
        pickle.dump( (new_errors, stein_errors, entry_errors), open(  'sphere_res_delta{0}_n{1}'.format(delta,n), 'wb' ) ) 

# function surface case

In [93]:
def exp_map(v, n ): 
    
    h = np.sum( v[0:int(n/2.)] ** 2 ) - np.sum( v[int(n/2.):] ** 2 )
    
    v = np.array( list(v) + [h] ) 
    
    return v 
    

## our method

In [94]:
def new_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    avg_est = 0 
    
    for _ in range(m): 

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        v = v / np.linalg.norm(v) 
        
        w = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
        w = w / np.linalg.norm(w) 
                
        f1 = func( exp_map( delta*v + delta*w, n ), noise_var, n + 1 ) 
        f2 = func( exp_map( delta*v - delta*w, n ), noise_var, n + 1 ) 
        f3 = func( exp_map( -delta*v + delta*w, n ), noise_var, n + 1 ) 
        f4 = func( exp_map( -delta*v - delta*w, n ), noise_var, n + 1 ) 

        grad_est = (f1 - f2 - f3 + f4) / ( 8 * delta**2 ) * n**2 * ( np.outer(v,w) + np.outer(w,v) ) 

        avg_est = avg_est + grad_est  

    avg_est = avg_est / m 
    
    return avg_est 

## Stein's method

In [95]:
def steins_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    avg_est = 0 
    
    for _ in range(m):

        v = np.random.multivariate_normal(np.zeros(n), np.identity(n) )
                
        f1 = func( exp_map( delta*v, n ), noise_var, n + 1 ) 
        f2 = func( exp_map( -delta*v, n ), noise_var, n + 1 ) 
        f3 = func( exp_map(np.zeros(n), n), noise_var, n + 1 ) 
        
        D2 = (f1 - 2*f3 + f2) / (2*delta**2) 

        grad_est = ( np.outer(v,v) - np.identity(n) ) * D2 

        avg_est = avg_est + grad_est 
        
    avg_est = avg_est / m 
    
    return avg_est

## entry-wise

In [96]:
def entry_est( func, n, noise_var, delta = 0.05 , m = 100 ):

    H_est = np.zeros((n,n))
    
    for i in range(n): 
        for j in range(n):
            
            avg_est = 0
            
            for _ in range(m):
                
                I = delta * np.array( [1 if k==i else 0 for k in range(n) ] )
                J = delta * np.array( [1 if k==j else 0 for k in range(n) ] )
            
                f1 = func( exp_map( I + J, n ) , noise_var, n + 1 ) 
                f2 = func( exp_map( I - J, n ), noise_var, n + 1 ) 
                f3 = func( exp_map( - I + J, n ), noise_var, n + 1 ) 
                f4 = func( exp_map( - I - J, n ), noise_var, n + 1 ) 

                hess_est = (f1 - f2 - f3 + f4) / ( 4 * delta**2 ) 
                
                avg_est = avg_est + hess_est  

            avg_est = avg_est / m 
            
            H_est[i,j] = avg_est 
    
    return H_est 

In [97]:
def res(n, delta, true_hess, func, noise_var = 1, m = 100, N = 100 ): 

    new_errors = [] 
    stein_errors = [] 
    entry_errors = [] 

    for _ in range(N): 
        
        new_errors.append( np.linalg.norm( new_est( func, n, noise_var, delta, int(m / 4. ) ) - true_hess , ord = 2 ) )
        
        stein_errors.append( np.linalg.norm( steins_est( func, n, noise_var, delta/np.sqrt(n), int(m/3.) ) - true_hess , ord = 2 ) )
        
        entry_errors.append( np.linalg.norm( entry_est( func, n, noise_var, delta, int(m/4./n/n) ) - true_hess , ord = 2 ) )
            
    return new_errors, stein_errors, entry_errors  

In [98]:
for n in Ns:
    
    for delta in DELTAs: 
    
        true_hess = entry_est( func1, n, 0, delta = 0.001 , m = 1 ) 

        new_errors, stein_errors, entry_errors = res(n , delta, true_hess, func1, noise_var = NOISE_VAR, m = SAMPLE_SIZE, N = 30 ) 
                  
        pickle.dump( (new_errors, stein_errors, entry_errors), open(  'hyperbolic_res_delta{0}_n{1}'.format(delta,n), 'wb' ) ) 